# Citation: Keras Model Code from [tensorflow.org](https://www.tensorflow.org/tutorials/keras/basic_text_classification) text classification
Testing out data preprocessing and mdoels for text analysis.

In [409]:
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import keras

In [410]:
########## Read in the datafile
df = pd.read_csv("suggestions_data_balanced.csv", engine = 'python');
df.fillna(-1, inplace=True)

print(df.dtypes)

is_prompt_exists     int64
comments            object
dtype: object


In [411]:
####### Declare constants
num_data = 10

In [412]:
###################### Create 
def bagofwords():
#     sentence = []
#     sentence.append("Hello there my name is name is blank.")
#     sentence.append("This is another sample name there phrase.")
#     text = nltk.word_tokenize(sentence[0].lower())
#     words = list(set(text))
#     text = nltk.word_tokenize(sentence[1].lower())
#     words.append(list(set(text)))
#     print(words)
    length = 0
    a = []
    for loop in range(num_data):
        comment = df['comments'][loop]
        sentTok = nltk.sent_tokenize(comment)
        length = length + len(sentTok)
        for sentence in sentTok:
    #         a.append("Hello there my name is name is blank.")
    #         a.append("This is another sample name there phrase.")
            a.append(sentence)
    
    
    a = (' '.join(a)).lower()
#     print(a)
#     print('\n')
    text = nltk.word_tokenize(a)
#     print(text)
    newset = set()
    for word in text:
#         print(word)
        newset.add(word)
#     print(newset)
#     print('\n\n')
    # Fill in found words
    word_to_ix = { w:(i+4) for i,w in enumerate(sorted(newset)) }
    ix_to_word = { (i+4):w for i,w in enumerate(sorted(newset)) }
    # Fill in reserved values
    ix_to_word[0] = "<PAD>"
    ix_to_word[1] = "<START>"
    ix_to_word[2] = "<UNK>"
    ix_to_word[3] = "<UNUSED>"
    word_to_ix["<PAD>"] = 0 # Used to equalize text length
    word_to_ix["<START>"] = 1
    word_to_ix["<UNK>"] = 2  # unknown value
    word_to_ix["<UNUSED>"] = 3
    return word_to_ix, ix_to_word
# print("\nLENGTH:", length)

In [413]:
word_to_ix, ix_to_word = bagofwords()
print(ix_to_word)
print('\n')
print(word_to_ix)

{4: ',', 5: '.', 6: '1.', 7: '?', 8: 'added', 9: 'aids', 10: 'along', 11: 'although', 12: 'an', 13: 'and', 14: 'appears', 15: 'are', 16: 'author', 17: 'be', 18: 'been', 19: 'bullet', 20: 'but', 21: 'calculate', 22: 'changes', 23: 'clearly', 24: 'code', 25: 'coding', 26: 'conventions', 27: 'design', 28: 'details', 29: 'diagrams', 30: 'document', 31: 'elaborately', 32: 'explain', 33: 'explained', 34: 'explains', 35: 'flow', 36: 'following', 37: 'follows', 38: 'functionality', 39: 'good', 40: 'has', 41: 'have', 42: 'how', 43: 'i', 44: 'images', 45: 'implementation', 46: 'in', 47: 'information', 48: 'is', 49: 'make', 50: 'mentioned', 51: 'method', 52: 'more', 53: "n't", 54: 'needed', 55: 'new', 56: 'no', 57: 'of', 58: 'on', 59: 'pattern', 60: 'patterns', 61: 'plan', 62: 'points', 63: 'pretty', 64: 'principles', 65: 'problem', 66: 'proposed', 67: 'provided', 68: 'ruby', 69: 'score', 70: 'seems', 71: 'should', 72: 'solving', 73: 'sound', 74: 'standards', 75: 'straightforward', 76: 'tests', 7

In [414]:
print(ix_to_word[3])
print(word_to_ix["straightforward"])

<UNUSED>
75


In [415]:
###################### Convert original data to number representations
loop = 0
for loop in range(num_data):
    comment = df['comments'][loop]
    comment = comment.lower()
    text = nltk.word_tokenize(comment)
    length = len(text)
    for i in range(length):
        text[i] = word_to_ix[text[i]]
#         print(text[i]) 
#     print(text)
    df['comments'][loop] = text

/Users/gabrielfinnrzingle/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [416]:
##################### Convert words back to check conversion 
for i in range(num_data):
    comment = df['comments'][i]
    length = len(comment)
    count = 0;
    text = []
    for num in comment:
        text.append(ix_to_word[num])
        count += 1
    text = (' '.join(text))
    print(text,'\n')

the changes mentioned are pretty straightforward , more information on how their plan to make those changes are should be provided in the document . 

the images and visual aids explain the flow very well . there were no uml diagrams in the document though . 

although , the writeup explains the functionality very well , they have n't mentioned which design pattern they have used an why ? . 

no new tests have been added 

changes are very good , the author has proposed design patterns they will be used in the implementation , along with that method to calculate score has been explained elaborately 

the design appears to be sound , but i think more details are needed . 

1. the plan is sound and seems to be clearly explained bullet points 

the principles used to solving the problem are sound 

yes , the code is well written and follows the conventions of ruby design principles . 

code written following the coding standards 



In [417]:
############# Initialize training data
train_data = df['comments'].head(8)
for i in range(8):
    print(train_data[i])

[78, 22, 50, 15, 63, 75, 4, 52, 47, 58, 42, 79, 61, 85, 49, 83, 22, 15, 71, 17, 67, 46, 78, 30, 5]
[78, 44, 13, 89, 9, 32, 78, 35, 88, 90, 5, 80, 91, 56, 86, 29, 46, 78, 30, 84, 5]
[11, 4, 78, 96, 34, 78, 38, 88, 90, 4, 81, 41, 53, 50, 92, 27, 59, 81, 41, 87, 12, 93, 7, 5]
[56, 55, 76, 41, 18, 8]
[22, 15, 88, 39, 4, 78, 16, 40, 66, 27, 60, 81, 94, 17, 87, 46, 78, 45, 4, 10, 95, 77, 51, 85, 21, 69, 40, 18, 33, 31]
[78, 27, 14, 85, 17, 73, 4, 20, 43, 82, 52, 28, 15, 54, 5]
[6, 78, 61, 48, 73, 13, 70, 85, 17, 23, 33, 19, 62]
[78, 64, 87, 85, 72, 78, 65, 15, 73]


In [418]:
############# Initialize testing data **(requires full dataset conversion for tail() to work)**
# print(df['comments'].tail(2))
# test_data = df['comments'].tail(2)
test_data = df['comments'].head(4) #### Remove later (PLACEHOLDER STATEMENT)
# for i in range(2):
#     print(test_data[i])

In [419]:
############ Function enabling conversion of indexed number sentence into word sentence
def decode_review(text):
#     return ' '.join([reverse_word_index.get(i, '?') for i in text])
    return ' '.join([ix_to_word[i] for i in text])

In [421]:
print(decode_review(train_data[0]))

the changes mentioned are pretty straightforward , more information on how their plan to make those changes are should be provided in the document .


In [426]:
############# Find the maximum sentence length to use for padding training data
maxlength = 0
for array in train_data:
    maxlength = max(len(array),maxlength)
print("Train:", maxlength)
for array in test_data:
    maxlength = max(len(array),maxlength)
print("Test:", maxlength)

Train: 30
Test: 30


In [ ]:
############# Initialize training labels
train_labels = df['is_prompt_exists'].head8

In [427]:
############# Pad the words to equalize array length
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                       value=word_to_ix["<PAD>"],
                                                       padding='post',
                                                       maxlen=maxlength)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                      value=word_to_ix["<PAD>"],
                                                      padding='post',
                                                      maxlen=maxlength)

In [428]:
len(train_data[0]), len(train_data[1]) # Check the new length of some train data

(30, 30)

In [429]:
print(train_data[0]) # Check new padded number sentences

[78 22 50 15 63 75  4 52 47 58 42 79 61 85 49 83 22 15 71 17 67 46 78 30
  5  0  0  0  0  0]


In [434]:
############# Creates the model
vocab_size = len(df) # suggestions_data_balanced.csv dataset length becomes input shape: 7272

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          116352    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 116,641
Trainable params: 116,641
Non-trainable params: 0
_________________________________________________________________


In [435]:
############ Configured model with optimizer and loss function
model.compile(optimizer = tf.train.AdamOptimizer(),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

In [436]:
########### Initiate validation sets
# Note -> vocab_size = len(df)
x_val = train_data[:vocab_size]
partial_x_train = train_data[vocab_size:]

y_val = train_labels[:vocab_size]
partial_y_train = train_labels[vocab_size:]

NameError: name 'train_labels' is not defined